# Globals

In [0]:
!pip install  area
import pdb
imgsFilePath = "gdrive/My Drive/projectA/Data/training_imagery/"
from google.colab import drive
drive.mount('/content/gdrive')
#for general class:
# testFilename="gdrive/My Drive/projectA/Data/test_w_n.csv"
# annotationFilename = "gdrive/My Drive/projectA/Data/train_w_n.csv"

#for subclass:
testFilename="gdrive/My Drive/projectA/Data/test_without_zoomout_large_vehicles.csv"
annotationFilename = "gdrive/My Drive/projectA/Data/train_without_zoomout_large_vehicles.csv"


In [0]:
def crop_rect(img, rect):
    # get the parameter of the small rectangle
    center = rect[0]
    size = rect[1]
    angle = rect[2]
    center, size = tuple(map(int, center)), tuple(map(int, size))

    # get row and col num in img
    height, width = img.shape[0], img.shape[1]

    M = cv2.getRotationMatrix2D(center, angle, 1)
    img_rot = cv2.warpAffine(img, M, (width, height))

    img_crop = cv2.getRectSubPix(img_rot, size, center)

    return img_crop, img_rot

#Big-Small

In [0]:
import glob
import numpy as np
import pandas as pd
import cv2
from google.colab.patches import cv2_imshow
from google.colab import drive
import os

from area import area
drive.mount('/content/gdrive')
label_map = {
    'bus':0, 'cement mixer': 1, 'crane truck': 2, 'dedicated agricultural vehicle': 3,
    'hatchback': 4, 'jeep':5, 'light truck':6, 'minibus':7, 'minivan':8, 'pickup':9,
    'prime mover': 10, 'sedan':11, 'tanker':12, 'truck': 13, 'van': 14
}

def prepareInputs(annotationFilename, imgsFilePath = "gdrive/My Drive/projectA/Data/training_imagery/",
                  sampleShape = (128,128), maxSetSize = 100000):
    print("Start preproccesing")
    annotationDB = pd.read_csv(annotationFilename)
    nRecords = annotationDB.shape[0]
    Samples = {};
    Labels = {};
    area_per_size = []; maxDistances =[]; i=0
    numOfSamples = 0
    invalids = 0  
    for recIndex in range(nRecords):
        if maxSetSize < 0 or numOfSamples < maxSetSize:
            imgID = annotationDB.loc[recIndex, 'image_id']
            objectBoundingBox = abs(np.array([[annotationDB.loc[recIndex, 'p1_x'], annotationDB.loc[recIndex, 'p_1y']],
                                          [annotationDB.loc[recIndex, 'p2_x'], annotationDB.loc[recIndex, 'p2_y']],
                                          [annotationDB.loc[recIndex, 'p3_x'], annotationDB.loc[recIndex, 'p3_y']],
                                          [annotationDB.loc[recIndex, 'p4_x'], annotationDB.loc[recIndex, 'p4_y']]]))
            imgFile = glob.glob(os.path.join(imgsFilePath, str(int(imgID)) + ".*"))
#             pdb.set_trace()
            if len(imgFile) == 1:
                img = cv2.imread(imgFile[0],cv2.IMREAD_COLOR)  # now img is 3-dim array
                cnt = objectBoundingBox.astype(int)
                patch = img[int(np.min(objectBoundingBox[:,1])):int(np.max(objectBoundingBox[:,1])),
                        int(np.min(objectBoundingBox[:,0])):int(np.max(objectBoundingBox[:,0]))]
                patchShape = patch.shape 
                if patchShape[0] > 0 and patchShape[1] > 0:
                    rect = cv2.minAreaRect(cnt)
                    box = cv2.boxPoints(rect)
                    box = np.int0(box)
                    cv2.drawContours(img, [box], 0, (0, 10, 0), 0)
                    img_crop, img_rot = crop_rect(img, rect)
                    patch = img_crop
                    patchShape =  patch.shape
                  
                    desired_size = sampleShape[0]
                    old_size = patchShape
                    ratio = float(sampleShape[0])/max(old_size)
                    new_size = tuple([int(x*ratio) for x in old_size])
                    old_sample = cv2.resize(patch, (new_size[1], new_size[0]))
                    delta_w = desired_size - new_size[1]
                    delta_h = desired_size - new_size[0]
                    top, bottom = delta_h//2, delta_h-(delta_h//2)
                    left, right = delta_w//2, delta_w-(delta_w//2)
                    color = [0, 0, 0]
                    sample = cv2.copyMakeBorder(old_sample, top, bottom, left, right, cv2.BORDER_CONSTANT,
                        value=color)
                    Samples[annotationDB.loc[recIndex, 'tag_id']] = sample
                    Labels[annotationDB.loc[recIndex, 'tag_id']] = label_map[annotationDB.loc[recIndex, 'sub_class']]
                    numOfSamples += 1
                    if Labels[annotationDB.loc[recIndex, 'tag_id']] not in [4,11]:
                      img_90 = np.rot90(sample)
                      new_tag =annotationDB.loc[recIndex, 'tag_id']*10+1
                      Samples[new_tag] = img_90
                      Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                      img_180 = np.rot90(img_90)
                      new_tag +=1
                      Samples[new_tag] = img_180
                      Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                      img_270 = np.rot90(img_180)
                      new_tag +=1
                      Samples[new_tag] = img_270
                      Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                      vertical_img = cv2.flip( sample, 1 )
                      new_tag +=1
                      Samples[new_tag] = vertical_img
                      Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                      vertical_img_90 = np.rot90(vertical_img)
                      new_tag +=1
                      Samples[new_tag] = vertical_img_90
                      Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                      vertical_img_180 = np.rot90(vertical_img_90)
                      new_tag +=1
                      Samples[new_tag] = vertical_img_180
                      Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                      vertical_img_270 = np.rot90(vertical_img_180)
                      new_tag +=1
                      Samples[new_tag] = vertical_img_270
                      Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                      numOfSamples += 7

                else:
                    print("Invalid image\n")
                    print (patchShape , "no. : " , recIndex , "tag: " , annotationDB.loc[recIndex, 'tag_id'])
                    invalids += 1
            else:
                pdb.set_trace()
                print("Image file missing!\n")
            i+=1
            if i%500 == 0:
              print("Finished image no. : " , i)
    print("Finished! valids: " , len(Samples) , "invalids: " , invalids)
    return [Samples, Labels, numOfSamples ]#, area_per_size,maxDistances]



In [0]:

def prepareInputs_test(annotationFilename, imgsFilePath = "gdrive/My Drive/projectA/Data/training_imagery/",
                  sampleShape = (128,128), maxSetSize = 10000):
    print("Start preproccesing")
    annotationDB = pd.read_csv(annotationFilename)
    nRecords = annotationDB.shape[0]
    Samples = {};
    Labels = {};
    area_per_size = []; maxDistances =[]; i=0
    numOfSamples = 0
    invalids = 0  
    for recIndex in range(nRecords):
        if maxSetSize < 0 or numOfSamples < maxSetSize:
            imgID = annotationDB.loc[recIndex, 'image_id']
            objectBoundingBox = abs(np.array([[annotationDB.loc[recIndex, 'p1_x'], annotationDB.loc[recIndex, 'p_1y']],
                                          [annotationDB.loc[recIndex, 'p2_x'], annotationDB.loc[recIndex, 'p2_y']],
                                          [annotationDB.loc[recIndex, 'p3_x'], annotationDB.loc[recIndex, 'p3_y']],
                                          [annotationDB.loc[recIndex, 'p4_x'], annotationDB.loc[recIndex, 'p4_y']]]))
            imgFile = glob.glob(os.path.join(imgsFilePath, str(int(imgID)) + ".*"))
            if len(imgFile) == 1:
                img = cv2.imread(imgFile[0],cv2.IMREAD_COLOR)  # now img is 3-dim array
                # TODO: check for image boundaries
                cnt = objectBoundingBox.astype(int)
                patch = img[int(np.min(objectBoundingBox[:,1])):int(np.max(objectBoundingBox[:,1])),
                        int(np.min(objectBoundingBox[:,0])):int(np.max(objectBoundingBox[:,0]))]
                patchShape = patch.shape 
                if patchShape[0] > 0 and patchShape[1] > 0:
                    rect = cv2.minAreaRect(cnt)
                    box = cv2.boxPoints(rect)
                    box = np.int0(box)
                    cv2.drawContours(img, [box], 0, (0, 10, 0), 0)
                    img_crop, img_rot = crop_rect(img, rect)
                    patch = img_crop
                    patchShape =  patch.shape
                    
                    desired_size = sampleShape[0]
                    old_size = patchShape
                    ratio = float(sampleShape[0])/max(old_size)
                    new_size = tuple([int(x*ratio) for x in old_size])
                    old_sample = cv2.resize(patch, (new_size[1], new_size[0]))
                    delta_w = desired_size - new_size[1]
                    delta_h = desired_size - new_size[0]
                    top, bottom = delta_h//2, delta_h-(delta_h//2)
                    left, right = delta_w//2, delta_w-(delta_w//2)
                    color = [0, 0, 0]
                    sample = cv2.copyMakeBorder(old_sample, top, bottom, left, right, cv2.BORDER_CONSTANT,
                        value=color)
                    Samples[annotationDB.loc[recIndex, 'tag_id']] = sample
                    Labels[annotationDB.loc[recIndex, 'tag_id']] = label_map[annotationDB.loc[recIndex, 'sub_class']]
                    numOfSamples += 1
                    

                else:
                    print("Invalid image\n")
                    print (patchShape , "no. : " , recIndex , "tag: " , annotationDB.loc[recIndex, 'tag_id'])
                    invalids += 1
            else:
                pdb.set_trace()
                print("Image file missing!\n")
            i+=1
            if i%500 == 0:
              print("Finished image no. : " , i)
    print("Finished! valids: " , len(Samples) , "invalids: " , invalids)
    return [Samples, Labels, numOfSamples ]#, area_per_size,maxDistances]

In [0]:
'''dataset preperation for subclss problem'''
import torch
import torchvision
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as data

class DataSet_train(data.Dataset):
    def __init__(self, filename):
        self.DB = pd.read_csv(filename)
        data, labels, numOfSamples = prepareInputs(filename)
        self.data = list(data.values())
        self.target = list(labels.values())
        self.n_samples = numOfSamples
    
    def __len__(self):   # Length of the dataset.
        return self.n_samples
    
    def __getitem__(self, index):   # Function that returns one point and one label.
        return torch.Tensor(self.data[index].copy()), self.target[index]
      
class DataSet_test(data.Dataset):
    def __init__(self, filename):
        self.DB = pd.read_csv(filename)
        data, labels, numOfSamples = prepareInputs_test(filename)
        self.data = list(data.values())
        self.target = list(labels.values())
        self.n_samples = numOfSamples
    
    def __len__(self):   # Length of the dataset.
        return self.n_samples
    
    def __getitem__(self, index):   # Function that returns one point and one label.
        return torch.Tensor(self.data[index].copy()), self.target[index]


In [0]:
my_data = DataSet_train(annotationFilename)
batch_size = 100
my_loader = data.DataLoader(my_data,batch_size=batch_size,shuffle=True,num_workers=1)

In [0]:
test_data = DataSet_test(testFilename)
test_loader = data.DataLoader(test_data,batch_size=batch_size,shuffle=False,num_workers=1)

In [0]:
'''model definition'''
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import torchvision.models as models
import torch.optim as optim
weights = torch.Tensor([1.,1.,1.,1.,3.,2.,2.,2.,2.,1.4,3.5,1.8,1.,1.4,2.])
weights = weights.to(device)
model = models.resnext50_32x4d(pretrained=True)
model.fc = nn.Linear(2048, 15)
model = model.to(device)
criterium = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=5e-4,weight_decay=1e-4)

In [0]:
test_loader2 = my_loader

In [0]:
'''Training'''
'''TODO: try train in balanced set by augment only big cars'''

for epoch in range(100):
  print('Starting epoch number: ',epoch)
  ################## evaluation (testing) section:     ############
  if epoch > 0 :
    cnt_of_success = np.zeros(15,)
    sums = np.zeros(15,)
    correct = 0
    total = 0
    num_of_larges = 0
    with torch.no_grad():
      for data in test_loader:
          images, labels = data
          images = images.permute(0,3,1,2)
          images = images.to(device)
          labels = labels.to(device)
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels.long().reshape(labels.__len__())).sum().item()
          for i in range(len(predicted)):
            sums[labels.long().reshape(labels.__len__())[i]]+=1
            cnt_of_success[predicted[i]]+=(predicted[i] == labels.long().reshape(labels.__len__())[i]).item()
            

    print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))
    print(correct , "  " , total)
    print(cnt_of_success/(sums+1e-7))
    print("predicted: ", cnt_of_success)
    print("sum of objects: ", sums)
    if (100 * correct / total) > 95:
      break
  ##################################################################
  
  ######################### Training section:     ##################
  for k, (data, label) in enumerate(my_loader):
      # Definition of inputs as variables for the net.
      # requires_grad is set False because we do not need to compute the 
      # derivative of the inputs
      data   = Variable(data,requires_grad=False)
      label = Variable(label.long(),requires_grad=False)
      data = data.permute(0,3,1,2)
      data = data.to(device)
      label = label.to(device)
      # Set gradient to 0.
      optimizer.zero_grad()
      # Feed forward.
      pred = model(data)
      pred = pred.to(device)
      # Loss calculation.
      loss = criterium(pred,label.view(-1))
      # Gradient calculation.
      loss.backward()

      # Print loss every 10 iterations.
      if k%100==0:
          print('Loss {:.4f} at iter {:d}'.format(loss.item(),k))
          

      # Model weight modification based on the optimizer. 
      optimizer.step()
   ##################################################################

# One Net - all features | large vehicles

In [0]:
def get_labels(annotationDB , recIndex):
  label = []
  label.append(np.maximum(annotationDB.loc[recIndex, 'open_cargo_area'], 0))
  label.append(np.maximum(annotationDB.loc[recIndex, 'wrecked'], 0))
  label.append(np.maximum(annotationDB.loc[recIndex, 'flatbed'], 0))
  label.append(np.maximum(annotationDB.loc[recIndex, 'ladder'], 0))
  label.append(np.maximum(annotationDB.loc[recIndex, 'enclosed_box'], 0))
  label.append(np.maximum(annotationDB.loc[recIndex, 'soft_shell_box'], 0))
  label.append(np.maximum(annotationDB.loc[recIndex, 'harnessed_to_a_cart'], 0))
  return label


testFilename="gdrive/My Drive/projectA/Data/test_without_zoomout_large_vehicles.csv"
annotationFilename = "gdrive/My Drive/projectA/Data/train_without_zoomout_large_vehicles.csv"

def prepareInputs(annotationFilename, imgsFilePath = "gdrive/My Drive/projectA/Data/training_imagery/",
                  sampleShape = (128,128), maxSetSize = 10000):
    print("Start preproccesing")
    annotationDB = pd.read_csv(annotationFilename)
    nRecords = annotationDB.shape[0]
    Samples = {};
    Labels = {};
    area_per_size = []; maxDistances =[]; i=0
    numOfSamples = 0
    invalids = 0  
    for recIndex in range(nRecords):
        if maxSetSize < 0 or numOfSamples < maxSetSize:
            imgID = annotationDB.loc[recIndex, 'image_id']
            objectBoundingBox = (np.array([[annotationDB.loc[recIndex, 'p1_x'], annotationDB.loc[recIndex, 'p_1y']],
                                          [annotationDB.loc[recIndex, 'p2_x'], annotationDB.loc[recIndex, 'p2_y']],
                                          [annotationDB.loc[recIndex, 'p3_x'], annotationDB.loc[recIndex, 'p3_y']],
                                          [annotationDB.loc[recIndex, 'p4_x'], annotationDB.loc[recIndex, 'p4_y']]]))
            for i,point in enumerate(objectBoundingBox):
              for j,coord in enumerate(point):
                if coord < 0:
                  objectBoundingBox[i][j] = 0
                  
            imgFile = glob.glob(os.path.join(imgsFilePath, str(int(imgID)) + ".*"))
            if len(imgFile) == 1:
                img = cv2.imread(imgFile[0],cv2.IMREAD_COLOR)  # now img is 3-dim array
                # TODO: check for image boundaries
                cnt = objectBoundingBox.astype(int)
                patch = img[int(np.min(objectBoundingBox[:,1])):int(np.max(objectBoundingBox[:,1])),
                        int(np.min(objectBoundingBox[:,0])):int(np.max(objectBoundingBox[:,0]))]
                patchShape = patch.shape 
                if patchShape[0] > 0 and patchShape[1] > 0:
                    rect = cv2.minAreaRect(cnt)
                    box = cv2.boxPoints(rect)
                    box = np.int0(box)
                    cv2.drawContours(img, [box], 0, (0, 10, 0), 0)
                    img_crop, img_rot = crop_rect(img, rect)
                    patch = img_crop
                    patchShape =  patch.shape
                    
                    desired_size = sampleShape[0]
                    old_size = patchShape
                    ratio = float(sampleShape[0])/max(old_size)
                    new_size = tuple([int(x*ratio) for x in old_size])
                    old_sample = cv2.resize(patch, (new_size[1], new_size[0]))
                    delta_w = desired_size - new_size[1]
                    delta_h = desired_size - new_size[0]
                    top, bottom = delta_h//2, delta_h-(delta_h//2)
                    left, right = delta_w//2, delta_w-(delta_w//2)
                    color = [0, 0, 0]
                    sample = cv2.copyMakeBorder(old_sample, top, bottom, left, right, cv2.BORDER_CONSTANT,
                        value=color)
                    Samples[annotationDB.loc[recIndex, 'tag_id']] = sample
                    
                    Labels[annotationDB.loc[recIndex, 'tag_id']] = get_labels(annotationDB , recIndex)
                    numOfSamples += 1
                    
                    if annotationDB.loc[recIndex, 'general_class'] == 1:
                      img_90 = np.rot90(sample)
                      new_tag =annotationDB.loc[recIndex, 'tag_id']*10+1
                      Samples[new_tag] = img_90
                      Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                      img_180 = np.rot90(img_90)
                      new_tag +=1
                      Samples[new_tag] = img_180
                      Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                      img_270 = np.rot90(img_180)
                      new_tag +=1
                      Samples[new_tag] = img_270
                      Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                      vertical_img = cv2.flip( sample, 1 )
                      new_tag +=1
                      Samples[new_tag] = vertical_img
                      Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                      vertical_img_90 = np.rot90(vertical_img)
                      new_tag +=1
                      Samples[new_tag] = vertical_img_90
                      Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                      vertical_img_180 = np.rot90(vertical_img_90)
                      new_tag +=1
                      Samples[new_tag] = vertical_img_180
                      Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                      vertical_img_270 = np.rot90(vertical_img_180)
                      new_tag +=1
                      Samples[new_tag] = vertical_img_270
                      Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                      numOfSamples += 7

                else:
                    print("Invalid image\n")
                    print (patchShape , "no. : " , recIndex , "tag: " , annotationDB.loc[recIndex, 'tag_id'])
                    invalids += 1
            else:
                pdb.set_trace()
                print("Image file missing!\n")
            i+=1
            if i%300 == 0:
              print("Finished image no. : " , i)
    print("Finished! valids: " , len(Samples) , "invalids: " , invalids)
    return [Samples, Labels, numOfSamples ]#, area_per_size,maxDistances]

In [0]:
'''dataset preperation for subclss problem'''
import torch
import torchvision
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as data

class DataSet_train(data.Dataset):
    def __init__(self, filename):
        self.DB = pd.read_csv(filename)
        data, labels, numOfSamples = prepareInputs(filename)
        self.data = list(data.values())
        self.target = list(labels.values())
        self.n_samples = numOfSamples
    
    def __len__(self):   # Length of the dataset.
        return self.n_samples
    
    def __getitem__(self, index):   # Function that returns one point and one label.
        return torch.Tensor(self.data[index].copy()), torch.Tensor(self.target[index].copy())
      
class DataSet_test(data.Dataset):
    def __init__(self, filename):
        self.DB = pd.read_csv(filename)
        data, labels, numOfSamples = prepareInputs(filename)
        self.data = list(data.values())
        self.target = list(labels.values())
        self.n_samples = numOfSamples
    
    def __len__(self):   # Length of the dataset.
        return self.n_samples
    
    def __getitem__(self, index):   # Function that returns one point and one label.
        return torch.Tensor(self.data[index].copy()), torch.Tensor(self.target[index].copy())


In [0]:
my_data = DataSet_train(annotationFilename)
batch_size = 50
my_loader = data.DataLoader(my_data,batch_size=batch_size,shuffle=True,num_workers=1)

In [0]:
test_data = DataSet_test(testFilename)
batch_size = 50
test_loader = data.DataLoader(test_data,batch_size=batch_size,shuffle=False,num_workers=1)

In [0]:
'''model definition'''
torch.cuda.empty_cache()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import torchvision.models as models
import torch.optim as optim
model = torchvision.models.resnet50(pretrained=True)

model.fc = nn.Linear(2048, 7)

model = model.to(device)
criterium = nn.CrossEntropyLoss()
criterium = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=5e-4,weight_decay=1e-4)

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [0]:
'''Training'''
'''TODO: try train in balanced set by augment only big cars'''

torch.cuda.empty_cache()
for epoch in range(100):
  print('Starting epoch number: ',epoch)
  ################## evaluation (testing) section:     ############
  if epoch >= 0 :
    correct_tot = np.zeros(7,)
    total_tot = np.zeros(7,)
    correct_tot1 = np.zeros(7,)
    total_tot1 = np.zeros(7,)
    correct = 0
    total = 0
    correct_1 = 0
    total_1 = 0
    num_of_larges = 0
    with torch.no_grad():
      for data in test_loader:
          images, labels = data
          images = images.permute(0,3,1,2)
          images = images.to(device)
          labels = labels.to(device)
          outputs = model(images)
          predicted = torch.round(outputs.data+0.35)
          for i,line in enumerate(labels):
            for j,label in enumerate(line):
               total_tot[j] += 1
               correct_tot[j] += (predicted[i,j] == label).item()
               if label == 1:
                  total_tot1[j] += 1
                  correct_tot1[j] += (predicted[i,j] == label).item()
                  total_1 +=1
                  correct_1 += (predicted[i,j] == label).item()
          total += (labels.size(0) * labels.size(1) ) 
          correct += (predicted == labels).sum().item()            
    print('Accuracy of the network on the 1795 test images: %d %%' % (100 * correct / total))
    print(correct , "  " , total)
    print('Accuracy of the network on the 1 class only: %d %%' % (100 * correct_1 / total_1))
    print(correct_1 , "  " , total_1)
    if (correct + correct_1 > best) and ((100 * correct / total) >= 93 and  (100 * correct_1 / total_1) >= 87 ):
      best = correct + correct_1
      path = "gdrive/My Drive/projectA/Models/all_features_large_model_resnet"
      torch.save(model, path)
      print("model saved...")
    for i in range(7):
      print('Accuracy of the network on the ' ,i,' class: %d %%' % (100 * correct_tot[i] / float(total_tot[i] + 1e-7)))
      print(correct_tot[i] , "  " , total_tot[i])
      print('Accuracy of the network on the ' ,i,' class, only 1: %d %%' % (100 * correct_tot1[i] / float(total_tot1[i] + 1e-7)))
      print(correct_tot1[i] , "  " , total_tot1[i])

     
  ##################################################################

  ######################### Training section:     ##################
  for k, (data, label) in enumerate(my_loader):
      # Definition of inputs as variables for the net.
      # requires_grad is set False because we do not need to compute the 
      # derivative of the inputs
      data   = Variable(data,requires_grad=False)
      label = Variable(label,requires_grad=False)
      data = data.permute(0,3,1,2)
      data = data.to(device)
      label = label.to(device)
      # Set gradient to 0.
      optimizer.zero_grad()
      # Feed forward.
      pred = model(data)
      pred = pred.to(device)
#       print(pred)
#       print(label.long())
      # Loss calculation.
      loss = criterium(pred,label)
      # Gradient calculation.
      loss.backward()

      # Print loss every 10 iterations.
      if k%30==0:
          print('Loss {:.4f} at iter {:d}'.format(loss.item(),k))
          

      # Model weight modification based on the optimizer. 
      optimizer.step()
   ##################################################################

In [0]:
path_resnet = "gdrive/My Drive/projectA/Models/all_features_model_resnet"
model_resnet = torch.load(path)
path_densenet = "gdrive/My Drive/projectA/Models/all_features_model_densenet"
model_densenet = torch.load(path)
path_resnext = "gdrive/My Drive/projectA/Models/all_features_model_resnext"
model_resnext = torch.load(path)

In [0]:
correct_tot = np.zeros(12,)
total_tot = np.zeros(12,)
correct_tot1 = np.zeros(12,)
total_tot1 = np.zeros(12,)
correct = 0
total = 0
correct_1 = 0
total_1 = 0
num_of_larges = 0
with torch.no_grad():
      for data in test_loader:
          images, labels = data
          images = images.permute(0,3,1,2)
          images = images.to(device)
          labels = labels.to(device)
          outputs_resnet = torch.round(model_resnet(images).data + 0.473)
          outputs_resnext = torch.round(model_resnext(images).data + 0.478)
          outputs_densenet = torch.round(model_densenet(images).data + 0.476)
#           _, predicted = torch.max(outputs.data, 1)
          predicted = torch.round( (outputs_densenet + outputs_resnet + outputs_resnext)/float(3) )
          for i,line in enumerate(labels):
            for j,label in enumerate(line):
               total_tot[j] += 1
               correct_tot[j] += (predicted[i,j] == label).item()
               if label == 1:
                  total_tot1[j] += 1
                  correct_tot1[j] += (predicted[i,j] == label).item()
                  total_1 +=1
                  correct_1 += (predicted[i,j] == label).item()
          total += (labels.size(0) * labels.size(1) ) 
          correct += (predicted == labels).sum().item()            
#           print(predicted.sum() )
print('Accuracy of the network on the 1795 test images: %d %%' % (100 * correct / total))
print(correct , "  " , total)
print('Accuracy of the network on the 1 class only: %d %%' % (100 * correct_1 / total_1))
print(correct_1 , "  " , total_1)



# One Net - all features | small vehicles

In [0]:
def get_labels(annotationDB , recIndex):
  label = []
  label.append(np.maximum(annotationDB.loc[recIndex, 'sunroof'], 0))
  label.append(np.maximum(annotationDB.loc[recIndex, 'luggage_carrier'], 0))
  label.append(np.maximum(annotationDB.loc[recIndex, 'open_cargo_area'], 0))
  label.append(np.maximum(annotationDB.loc[recIndex, 'enclosed_cab'], 0))
  label.append(np.maximum(annotationDB.loc[recIndex, 'spare_wheel'], 0))
  label.append(np.maximum(annotationDB.loc[recIndex, 'wrecked'], 0))
  return label

testFilename="gdrive/My Drive/projectA/Data/test_without_zoomout_small_vehicles.csv"
annotationFilename = "gdrive/My Drive/projectA/Data/train_without_zoomout_small_vehicles.csv"

def prepareInputs(annotationFilename, imgsFilePath = "gdrive/My Drive/projectA/Data/training_imagery/",
                  sampleShape = (128,128), maxSetSize = 10000):
    print("Start preproccesing")
    annotationDB = pd.read_csv(annotationFilename)
    nRecords = annotationDB.shape[0]
    Samples = {};
    Labels = {};
    area_per_size = []; maxDistances =[]; i=0
    numOfSamples = 0
    invalids = 0  
    for recIndex in range(nRecords):
        if maxSetSize < 0 or numOfSamples < maxSetSize:
            imgID = annotationDB.loc[recIndex, 'image_id']
            objectBoundingBox = (np.array([[annotationDB.loc[recIndex, 'p1_x'], annotationDB.loc[recIndex, 'p_1y']],
                                          [annotationDB.loc[recIndex, 'p2_x'], annotationDB.loc[recIndex, 'p2_y']],
                                          [annotationDB.loc[recIndex, 'p3_x'], annotationDB.loc[recIndex, 'p3_y']],
                                          [annotationDB.loc[recIndex, 'p4_x'], annotationDB.loc[recIndex, 'p4_y']]]))
            for i,point in enumerate(objectBoundingBox):
              for j,coord in enumerate(point):
                if coord < 0:
                  objectBoundingBox[i][j] = 0
                  
            imgFile = glob.glob(os.path.join(imgsFilePath, str(int(imgID)) + ".*"))
            if len(imgFile) == 1:
                img = cv2.imread(imgFile[0],cv2.IMREAD_COLOR)  # now img is 3-dim array
                # TODO: check for image boundaries
                cnt = objectBoundingBox.astype(int)
                patch = img[int(np.min(objectBoundingBox[:,1])):int(np.max(objectBoundingBox[:,1])),
                        int(np.min(objectBoundingBox[:,0])):int(np.max(objectBoundingBox[:,0]))]
                patchShape = patch.shape 
                if patchShape[0] > 0 and patchShape[1] > 0:
                    rect = cv2.minAreaRect(cnt)
                    box = cv2.boxPoints(rect)
                    box = np.int0(box)
                    cv2.drawContours(img, [box], 0, (0, 10, 0), 0)
                    img_crop, img_rot = crop_rect(img, rect)
                    patch = img_crop
                    patchShape =  patch.shape
                    
                    desired_size = sampleShape[0]
                    old_size = patchShape
                    ratio = float(sampleShape[0])/max(old_size)
                    new_size = tuple([int(x*ratio) for x in old_size])
                    old_sample = cv2.resize(patch, (new_size[1], new_size[0]))
                    delta_w = desired_size - new_size[1]
                    delta_h = desired_size - new_size[0]
                    top, bottom = delta_h//2, delta_h-(delta_h//2)
                    left, right = delta_w//2, delta_w-(delta_w//2)
                    color = [0, 0, 0]
                    sample = cv2.copyMakeBorder(old_sample, top, bottom, left, right, cv2.BORDER_CONSTANT,
                        value=color)
                    Samples[annotationDB.loc[recIndex, 'tag_id']] = sample
                    
                    Labels[annotationDB.loc[recIndex, 'tag_id']] = get_labels(annotationDB , recIndex)
                    numOfSamples += 1
                    
                    if annotationDB.loc[recIndex, 'general_class'] == 1:
                      img_90 = np.rot90(sample)
                      new_tag =annotationDB.loc[recIndex, 'tag_id']*10+1
                      Samples[new_tag] = img_90
                      Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                      img_180 = np.rot90(img_90)
                      new_tag +=1
                      Samples[new_tag] = img_180
                      Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                      img_270 = np.rot90(img_180)
                      new_tag +=1
                      Samples[new_tag] = img_270
                      Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                      vertical_img = cv2.flip( sample, 1 )
                      new_tag +=1
                      Samples[new_tag] = vertical_img
                      Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                      vertical_img_90 = np.rot90(vertical_img)
                      new_tag +=1
                      Samples[new_tag] = vertical_img_90
                      Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                      vertical_img_180 = np.rot90(vertical_img_90)
                      new_tag +=1
                      Samples[new_tag] = vertical_img_180
                      Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                      vertical_img_270 = np.rot90(vertical_img_180)
                      new_tag +=1
                      Samples[new_tag] = vertical_img_270
                      Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                      numOfSamples += 7

                else:
                    print("Invalid image\n")
                    print (patchShape , "no. : " , recIndex , "tag: " , annotationDB.loc[recIndex, 'tag_id'])
                    invalids += 1
            else:
                pdb.set_trace()
                print("Image file missing!\n")
            i+=1
            if i%300 == 0:
              print("Finished image no. : " , i)
    print("Finished! valids: " , len(Samples) , "invalids: " , invalids)
    return [Samples, Labels, numOfSamples ]#, area_per_size,maxDistances]

In [0]:
'''dataset preperation for subclss problem'''
import torch
import torchvision
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as data

class DataSet_train(data.Dataset):
    def __init__(self, filename):
        self.DB = pd.read_csv(filename)
        data, labels, numOfSamples = prepareInputs(filename)
        self.data = list(data.values())
        self.target = list(labels.values())
        self.n_samples = numOfSamples
    
    def __len__(self):   # Length of the dataset.
        return self.n_samples
    
    def __getitem__(self, index):   # Function that returns one point and one label.
        return torch.Tensor(self.data[index].copy()), torch.Tensor(self.target[index].copy())
      
class DataSet_test(data.Dataset):
    def __init__(self, filename):
        self.DB = pd.read_csv(filename)
        data, labels, numOfSamples = prepareInputs(filename)
        self.data = list(data.values())
        self.target = list(labels.values())
        self.n_samples = numOfSamples
    
    def __len__(self):   # Length of the dataset.
        return self.n_samples
    
    def __getitem__(self, index):   # Function that returns one point and one label.
        return torch.Tensor(self.data[index].copy()), torch.Tensor(self.target[index].copy())


In [0]:
my_data = DataSet_train(annotationFilename)
batch_size = 100
my_loader = data.DataLoader(my_data,batch_size=batch_size,shuffle=True,num_workers=1)

In [0]:
test_data = DataSet_test(testFilename)
batch_size = 50
test_loader = data.DataLoader(test_data,batch_size=batch_size,shuffle=False,num_workers=1)

In [0]:
path_resnet = "gdrive/My Drive/projectA/Models/all_features_small_model_resnet_final"
model_resnet = torch.load(path_resnet)

In [0]:
'''model definition'''
torch.cuda.empty_cache()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import torchvision.models as models
import torch.optim as optim

model = torchvision.models.resnet50(pretrained=True)


model.fc = nn.Linear(2048, 6)
model = model.to(device)
criterium = nn.CrossEntropyLoss()
criterium = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=5e-4,weight_decay=1e-4)

In [0]:
torch.cuda.empty_cache()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [0]:
'''Training'''
'''TODO: try train in balanced set by augment only big cars'''
model = model_resnet
torch.cuda.empty_cache()
for epoch in range(100):
  print('Starting epoch number: ',epoch)
  ################## evaluation (testing) section:     ############
  if epoch >= 0 :
    correct_tot = np.zeros(6,)
    total_tot = np.zeros(6,)
    correct_tot1 = np.zeros(6,)
    total_tot1 = np.zeros(6,)
    correct = 0
    total = 0
    correct_1 = 0
    total_1 = 0
    num_of_larges = 0
    with torch.no_grad():
      for data in test_loader:
          images, labels = data
          images = images.permute(0,3,1,2)
          images = images.to(device)
          labels = labels.to(device)
          outputs = model(images)
          predicted = torch.round(outputs.data+0.2)
          for i,line in enumerate(labels):
            for j,label in enumerate(line):
               total_tot[j] += 1
               correct_tot[j] += (predicted[i,j] == label).item()
               if label == 1:
                  total_tot1[j] += 1
                  correct_tot1[j] += (predicted[i,j] == label).item()
                  total_1 +=1
                  correct_1 += (predicted[i,j] == label).item()
          total += (labels.size(0) * labels.size(1) ) 
          correct += (predicted == labels).sum().item()            
    print('Accuracy of the network on the 1795 test images: %d %%' % (100 * correct / total))
    print(correct , "  " , total)
    print('Accuracy of the network on the 1 class only: %d %%' % (100 * correct_1 / total_1))
    print(correct_1 , "  " , total_1)
    if (correct + correct_1 > best) and ((100 * correct / total) >= 93 and  (100 * correct_1 / total_1) >= 87 ):
      best = correct + correct_1
      path = "gdrive/My Drive/projectA/Models/all_features_small_model_resnet"
      torch.save(model, path)
      print("model saved...")
    for i in range(6):
      print('Accuracy of the network on the ' ,i,' class: %d %%' % (100 * correct_tot[i] / float(total_tot[i] + 1e-7)))
      print(correct_tot[i] , "  " , total_tot[i])
      print('Accuracy of the network on the ' ,i,' class, only 1: %d %%' % (100 * correct_tot1[i] / float(total_tot1[i] + 1e-7)))
      print(correct_tot1[i] , "  " , total_tot1[i])
    if correct > best:
      best = correct
      path = "gdrive/My Drive/projectA/Models/all_features_model"
      torch.save(model, path)
    if (100 * correct / total) > 95:
      break
    if (100 * correct / total) > 92 and (100 * correct_1 / total_1)>72:
      path = "gdrive/My Drive/projectA/Models/all_features_model_densenet"
      torch.save(model, path)
     
  ##################################################################
  break
  ######################### Training section:     ##################
  for k, (data, label) in enumerate(my_loader):
      # Definition of inputs as variables for the net.
      # requires_grad is set False because we do not need to compute the 
      # derivative of the inputs
      data   = Variable(data,requires_grad=False)
      label = Variable(label,requires_grad=False)
      data = data.permute(0,3,1,2)
      data = data.to(device)
      label = label.to(device)
      # Set gradient to 0.
      optimizer.zero_grad()
      # Feed forward.
      pred = model(data)
      pred = pred.to(device)
      # Loss calculation.
      loss = criterium(pred,label)
      # Gradient calculation.
      loss.backward()

      # Print loss every 10 iterations.
      if k%30==0:
          print('Loss {:.4f} at iter {:d}'.format(loss.item(),k))
          

      # Model weight modification based on the optimizer. 
      optimizer.step()
   ##################################################################

# sub_class | large vehicles

In [0]:
label_map = {
    'bus':0, 'cement mixer': 1, 'crane truck': 2, 'dedicated agricultural vehicle': 3,
    'light truck':4, 'minibus':5,
    'prime mover': 6, 'tanker':7, 'truck': 8
}

def get_labels(annotationDB , recIndex):
  label = annotationDB.loc[recIndex, 'sub_class']
  return label_map[label]


testFilename="gdrive/My Drive/projectA/Data/test_without_zoomout_large_vehicles.csv"
annotationFilename = "gdrive/My Drive/projectA/Data/train_without_zoomout_large_vehicles.csv"

def prepareInputs(annotationFilename, imgsFilePath = "gdrive/My Drive/projectA/Data/training_imagery/",
                  sampleShape = (128,128), maxSetSize = 10000):
    print("Start preproccesing")
    annotationDB = pd.read_csv(annotationFilename)
    nRecords = annotationDB.shape[0]
    Samples = {};
    Labels = {};
    area_per_size = []; maxDistances =[]; i=0
    numOfSamples = 0
    invalids = 0  
    for recIndex in range(nRecords):
        if maxSetSize < 0 or numOfSamples < maxSetSize:
            imgID = annotationDB.loc[recIndex, 'image_id']
            objectBoundingBox = (np.array([[annotationDB.loc[recIndex, 'p1_x'], annotationDB.loc[recIndex, 'p_1y']],
                                          [annotationDB.loc[recIndex, 'p2_x'], annotationDB.loc[recIndex, 'p2_y']],
                                          [annotationDB.loc[recIndex, 'p3_x'], annotationDB.loc[recIndex, 'p3_y']],
                                          [annotationDB.loc[recIndex, 'p4_x'], annotationDB.loc[recIndex, 'p4_y']]]))
            for i,point in enumerate(objectBoundingBox):
              for j,coord in enumerate(point):
                if coord < 0:
                  objectBoundingBox[i][j] = 0
                  
            imgFile = glob.glob(os.path.join(imgsFilePath, str(int(imgID)) + ".*"))
            if len(imgFile) == 1:
                img = cv2.imread(imgFile[0],cv2.IMREAD_COLOR)  # now img is 3-dim array
                cnt = objectBoundingBox.astype(int)
                patch = img[int(np.min(objectBoundingBox[:,1])):int(np.max(objectBoundingBox[:,1])),
                        int(np.min(objectBoundingBox[:,0])):int(np.max(objectBoundingBox[:,0]))]
                patchShape = patch.shape 
                if patchShape[0] > 0 and patchShape[1] > 0:
                    rect = cv2.minAreaRect(cnt)
                    box = cv2.boxPoints(rect)
                    box = np.int0(box)
                    cv2.drawContours(img, [box], 0, (0, 10, 0), 0)
                    img_crop, img_rot = crop_rect(img, rect)
                    patch = img_crop
                    patchShape =  patch.shape
                    
                    desired_size = sampleShape[0]
                    old_size = patchShape
                    ratio = float(sampleShape[0])/max(old_size)
                    new_size = tuple([int(x*ratio) for x in old_size])
                    old_sample = cv2.resize(patch, (new_size[1], new_size[0]))
                    delta_w = desired_size - new_size[1]
                    delta_h = desired_size - new_size[0]
                    top, bottom = delta_h//2, delta_h-(delta_h//2)
                    left, right = delta_w//2, delta_w-(delta_w//2)
                    color = [0, 0, 0]
                    sample = cv2.copyMakeBorder(old_sample, top, bottom, left, right, cv2.BORDER_CONSTANT,
                        value=color)
                    Samples[annotationDB.loc[recIndex, 'tag_id']] = sample
                    
                    Labels[annotationDB.loc[recIndex, 'tag_id']] = get_labels(annotationDB , recIndex)
                    numOfSamples += 1
                    
                    if annotationDB.loc[recIndex, 'general_class'] == 1:
                      img_90 = np.rot90(sample)
                      new_tag =annotationDB.loc[recIndex, 'tag_id']*10+1
                      Samples[new_tag] = img_90
                      Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                      img_180 = np.rot90(img_90)
                      new_tag +=1
                      Samples[new_tag] = img_180
                      Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                      img_270 = np.rot90(img_180)
                      new_tag +=1
                      Samples[new_tag] = img_270
                      Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                      vertical_img = cv2.flip( sample, 1 )
                      new_tag +=1
                      Samples[new_tag] = vertical_img
                      Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                      vertical_img_90 = np.rot90(vertical_img)
                      new_tag +=1
                      Samples[new_tag] = vertical_img_90
                      Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                      vertical_img_180 = np.rot90(vertical_img_90)
                      new_tag +=1
                      Samples[new_tag] = vertical_img_180
                      Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                      vertical_img_270 = np.rot90(vertical_img_180)
                      new_tag +=1
                      Samples[new_tag] = vertical_img_270
                      Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                      numOfSamples += 7

                else:
                    print("Invalid image\n")
                    print (patchShape , "no. : " , recIndex , "tag: " , annotationDB.loc[recIndex, 'tag_id'])
                    invalids += 1
            else:
                pdb.set_trace()
                print("Image file missing!\n")
            i+=1
            if i%300 == 0:
              print("Finished image no. : " , i)
    print("Finished! valids: " , len(Samples) , "invalids: " , invalids)
    return [Samples, Labels, numOfSamples ]#, area_per_size,maxDistances]

In [0]:
'''dataset preperation for subclss problem'''
import torch
import torchvision
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as data

class DataSet_train(data.Dataset):
    def __init__(self, filename):
        self.DB = pd.read_csv(filename)
        data, labels, numOfSamples = prepareInputs(filename)
        self.data = list(data.values())
        self.target = list(labels.values())
        self.n_samples = numOfSamples
    
    def __len__(self):   # Length of the dataset.
        return self.n_samples
    
    def __getitem__(self, index):   # Function that returns one point and one label.
        return torch.Tensor(self.data[index].copy()),self.target[index]
      
class DataSet_test(data.Dataset):
    def __init__(self, filename):
        self.DB = pd.read_csv(filename)
        data, labels, numOfSamples = prepareInputs(filename)
        self.data = list(data.values())
        self.target = list(labels.values())
        self.n_samples = numOfSamples
    
    def __len__(self):   # Length of the dataset.
        return self.n_samples
    
    def __getitem__(self, index):   # Function that returns one point and one label.
        return torch.Tensor(self.data[index].copy()), self.target[index]


In [0]:
my_data = DataSet_train(annotationFilename)
batch_size = 50
my_loader = data.DataLoader(my_data,batch_size=batch_size,shuffle=True,num_workers=1)

In [0]:
test_data = DataSet_test(testFilename)
batch_size = 50
test_loader = data.DataLoader(test_data,batch_size=batch_size,shuffle=False,num_workers=1)

In [0]:
'''model definition'''
torch.cuda.empty_cache()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import torchvision.models as models
import torch.optim as optim
model = torchvision.models.resnet152(pretrained=True)

model.fc = nn.Linear(2048, 9)
model = model.to(device)
criterium = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=5e-4,weight_decay=1e-4)

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [0]:
'''Training'''
'''TODO: try train in balanced set by augment only big cars'''
torch.cuda.empty_cache()
for epoch in range(100):
  print('Starting epoch number: ',epoch)
  ################## evaluation (testing) section:     ############
  if epoch >= 0 :
    cnt_of_success = np.zeros(9,)
    sums = np.zeros(9,)
    correct = 0
    total = 0
    num_of_larges = 0
    with torch.no_grad():
      for data in test_loader:
          images, labels = data
          images = images.permute(0,3,1,2)
          images = images.to(device)
          labels = labels.to(device)
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels.long().reshape(labels.__len__())).sum().item()
          for i in range(len(predicted)):
            sums[labels.long().reshape(labels.__len__())[i]]+=1
            cnt_of_success[predicted[i]]+=(predicted[i] == labels.long().reshape(labels.__len__())[i]).item()
            

    print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))
    print(correct , "  " , total)
    if correct > best:
      best = correct
      path = "gdrive/My Drive/projectA/Models/sub_class_large_model_resnet"
      torch.save(model, path)
      print("model saved...")

    if correct > best:
      best = correct
      path = "gdrive/My Drive/projectA/Models/all_features_model"
      torch.save(model, path)
    if (100 * correct / total) > 95:
      break
    if (100 * correct / total) > 92 and (100 * correct_1 / total_1)>72:
      path = "gdrive/My Drive/projectA/Models/all_features_model_densenet"
      torch.save(model, path)
     
  ##################################################################

  ######################### Training section:     ##################
  for k, (data, label) in enumerate(my_loader):
      # Definition of inputs as variables for the net.
      # requires_grad is set False because we do not need to compute the 
      # derivative of the inputs
      data   = Variable(data,requires_grad=False)
      label = Variable(label,requires_grad=False)
      data = data.permute(0,3,1,2)
      data = data.to(device)
      label = label.to(device)
      # Set gradient to 0.
      optimizer.zero_grad()
      # Feed forward.
      pred = model(data)
      pred = pred.to(device)

      # Loss calculation.
      loss = criterium(pred,label.view(-1))
      # Gradient calculation.
      loss.backward()

      # Print loss every 10 iterations.
      if k%30==0:
          print('Loss {:.4f} at iter {:d}'.format(loss.item(),k))
          

      # Model weight modification based on the optimizer. 
      optimizer.step()
   ##################################################################

In [0]:
path_resnet = "gdrive/My Drive/projectA/Models/all_features_model_resnet"
model_resnet = torch.load(path)
path_densenet = "gdrive/My Drive/projectA/Models/all_features_model_densenet"
model_densenet = torch.load(path)
path_resnext = "gdrive/My Drive/projectA/Models/all_features_model_resnext"
model_resnext = torch.load(path)

In [0]:
correct_tot = np.zeros(12,)
total_tot = np.zeros(12,)
correct_tot1 = np.zeros(12,)
total_tot1 = np.zeros(12,)
correct = 0
total = 0
correct_1 = 0
total_1 = 0
num_of_larges = 0
with torch.no_grad():
      for data in test_loader:
          images, labels = data
          images = images.permute(0,3,1,2)
          images = images.to(device)
          labels = labels.to(device)
          outputs_resnet = torch.round(model_resnet(images).data + 0.473)
          outputs_resnext = torch.round(model_resnext(images).data + 0.478)
          outputs_densenet = torch.round(model_densenet(images).data + 0.476)
#           _, predicted = torch.max(outputs.data, 1)
          predicted = torch.round( (outputs_densenet + outputs_resnet + outputs_resnext)/float(3) )
          for i,line in enumerate(labels):
            for j,label in enumerate(line):
               total_tot[j] += 1
               correct_tot[j] += (predicted[i,j] == label).item()
               if label == 1:
                  total_tot1[j] += 1
                  correct_tot1[j] += (predicted[i,j] == label).item()
                  total_1 +=1
                  correct_1 += (predicted[i,j] == label).item()
          total += (labels.size(0) * labels.size(1) ) 
          correct += (predicted == labels).sum().item()            
#           print(predicted.sum() )
print('Accuracy of the network on the 1795 test images: %d %%' % (100 * correct / total))
print(correct , "  " , total)
print('Accuracy of the network on the 1 class only: %d %%' % (100 * correct_1 / total_1))
print(correct_1 , "  " , total_1)



# sub_class | small vehicles

In [0]:
label_map = {
    'hatchback': 0, 'jeep':1, 'minivan':2, 'pickup':3,
    'sedan':4, 'van': 5
}

def get_labels(annotationDB , recIndex):
  label = annotationDB.loc[recIndex, 'sub_class']
  return label_map[label]


testFilename="gdrive/My Drive/projectA/Data/red_green.csv"
annotationFilename = "gdrive/My Drive/projectA/Data/train_without_zoomout_small_vehicles.csv"

def prepareInputs(annotationFilename, imgsFilePath = "gdrive/My Drive/projectA/Data/training_imagery/",
                  sampleShape = (128,128), maxSetSize = 10000):
    print("Start preproccesing")
    annotationDB = pd.read_csv(annotationFilename)
    nRecords = annotationDB.shape[0]
    Samples = {};
    Labels = {};
    area_per_size = []; maxDistances =[]; i=0
    numOfSamples = 0
    invalids = 0  
    for recIndex in range(nRecords):
        if annotationDB.loc[recIndex, 'general_class'] == 1:
          continue
        if maxSetSize < 0 or numOfSamples < maxSetSize:
            imgID = annotationDB.loc[recIndex, 'image_id']
            objectBoundingBox = (np.array([[annotationDB.loc[recIndex, 'p1_x'], annotationDB.loc[recIndex, 'p_1y']],
                                          [annotationDB.loc[recIndex, 'p2_x'], annotationDB.loc[recIndex, 'p2_y']],
                                          [annotationDB.loc[recIndex, 'p3_x'], annotationDB.loc[recIndex, 'p3_y']],
                                          [annotationDB.loc[recIndex, 'p4_x'], annotationDB.loc[recIndex, 'p4_y']]]))
            for i,point in enumerate(objectBoundingBox):
              for j,coord in enumerate(point):
                if coord < 0:
                  objectBoundingBox[i][j] = 0
                  
            imgFile = glob.glob(os.path.join(imgsFilePath, str(int(imgID)) + ".*"))
            if len(imgFile) == 1:
                img = cv2.imread(imgFile[0],cv2.IMREAD_COLOR)  # now img is 3-dim array
                cnt = objectBoundingBox.astype(int)
                patch = img[int(np.min(objectBoundingBox[:,1])):int(np.max(objectBoundingBox[:,1])),
                        int(np.min(objectBoundingBox[:,0])):int(np.max(objectBoundingBox[:,0]))]
                patchShape = patch.shape 
                if patchShape[0] > 0 and patchShape[1] > 0:
                    rect = cv2.minAreaRect(cnt)
                    box = cv2.boxPoints(rect)
                    box = np.int0(box)
                    cv2.drawContours(img, [box], 0, (0, 10, 0), 0)
                    img_crop, img_rot = crop_rect(img, rect)
                    patch = img_crop
                    patchShape =  patch.shape
                    
                    desired_size = sampleShape[0]
                    old_size = patchShape
                    ratio = float(sampleShape[0])/max(old_size)
                    new_size = tuple([int(x*ratio) for x in old_size])
                    old_sample = cv2.resize(patch, (new_size[1], new_size[0]))
                    delta_w = desired_size - new_size[1]
                    delta_h = desired_size - new_size[0]
                    top, bottom = delta_h//2, delta_h-(delta_h//2)
                    left, right = delta_w//2, delta_w-(delta_w//2)
                    color = [0, 0, 0]
                    sample = cv2.copyMakeBorder(old_sample, top, bottom, left, right, cv2.BORDER_CONSTANT,
                        value=color)
                    Samples[annotationDB.loc[recIndex, 'tag_id']] = sample
                    
                    Labels[annotationDB.loc[recIndex, 'tag_id']] = get_labels(annotationDB , recIndex)
                    numOfSamples += 1
                    
                else:
                    print("Invalid image\n")
                    print (patchShape , "no. : " , recIndex , "tag: " , annotationDB.loc[recIndex, 'tag_id'])
                    invalids += 1
            else:
                pdb.set_trace()
                print("Image file missing!\n")
            i+=1
            if i%30 == 0:
              print("Finished image no. : " , i)
    print("Finished! valids: " , len(Samples) , "invalids: " , invalids)
    return [Samples, Labels, numOfSamples ]

In [0]:
'''dataset preperation for subclss problem'''
import torch
import torchvision
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as data

class DataSet_train(data.Dataset):
    def __init__(self, filename):
        self.DB = pd.read_csv(filename)
        data, labels, numOfSamples = prepareInputs(filename)
        self.data = list(data.values())
        self.target = list(labels.values())
        self.n_samples = numOfSamples
    
    def __len__(self):   # Length of the dataset.
        return self.n_samples
    
    def __getitem__(self, index):   # Function that returns one point and one label.
        return torch.Tensor(self.data[index].copy()),self.target[index]
      
class DataSet_test(data.Dataset):
    def __init__(self, filename):
        self.DB = pd.read_csv(filename)
        data, labels, numOfSamples = prepareInputs(filename)
        self.data = list(data.values())
        self.target = list(labels.values())
        self.n_samples = numOfSamples
    
    def __len__(self):   # Length of the dataset.
        return self.n_samples
    
    def __getitem__(self, index):   # Function that returns one point and one label.
        return torch.Tensor(self.data[index].copy()), self.target[index]


In [0]:
my_data = DataSet_train(annotationFilename)
batch_size = 50
my_loader = data.DataLoader(my_data,batch_size=batch_size,shuffle=True,num_workers=1)

In [0]:
test_data = DataSet_test(testFilename)
batch_size = 50
test_loader = data.DataLoader(test_data,batch_size=batch_size,shuffle=False,num_workers=1)

In [0]:
'''model definition'''
torch.cuda.empty_cache()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import torchvision.models as models
import torch.optim as optim
model = torchvision.models.resnet50(pretrained=True)

model.fc = nn.Linear(2048, 6)
model = model.to(device)
criterium = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=5e-4,weight_decay=1e-4)

In [0]:
path_resnet = "gdrive/My Drive/projectA/Models/sub_class_small_model_resnet_final"
model = torch.load(path_resnet)

In [0]:
'''Training'''
'''TODO: try train in balanced set by augment only big cars'''
torch.cuda.empty_cache()
for epoch in range(100):
  print('Starting epoch number: ',epoch)
  ################## evaluation (testing) section:     ############
  if epoch >= 0 :
    cnt_of_success = np.zeros(6,)
    sums = np.zeros(6,)
    correct = 0
    total = 0
    num_of_larges = 0
    with torch.no_grad():
      for data in test_loader:
          images, labels = data
          images = images.permute(0,3,1,2)
          images = images.to(device)
          labels = labels.to(device)
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          print (predicted)
          total += labels.size(0)
          correct += (predicted == labels.long().reshape(labels.__len__())).sum().item()
          for i in range(len(predicted)):
            sums[labels.long().reshape(labels.__len__())[i]]+=1
            cnt_of_success[predicted[i]]+=(predicted[i] == labels.long().reshape(labels.__len__())[i]).item()
            

    print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))
    print(correct , "  " , total)
    if correct > best:
      best = correct
      path = "gdrive/My Drive/projectA/Models/sub_class_small_model_resnet"
      torch.save(model, path)
      print("model saved...")

    if correct > best:
      best = correct
      path = "gdrive/My Drive/projectA/Models/all_features_model"
      torch.save(model, path)
    if (100 * correct / total) > 95:
      break
    if (100 * correct / total) > 92 and (100 * correct_1 / total_1)>72:
      path = "gdrive/My Drive/projectA/Models/all_features_model_densenet"
      torch.save(model, path)
     
  ##################################################################
  break
  ######################### Training section:     ##################
  for k, (data, label) in enumerate(my_loader):
      # Definition of inputs as variables for the net.
      # requires_grad is set False because we do not need to compute the 
      # derivative of the inputs
      data   = Variable(data,requires_grad=False)
      label = Variable(label,requires_grad=False)
      data = data.permute(0,3,1,2)
      data = data.to(device)
      label = label.to(device)
      # Set gradient to 0.
      optimizer.zero_grad()
      # Feed forward.
      pred = model(data)
      pred = pred.to(device)
      # Loss calculation.
      loss = criterium(pred,label.view(-1))
      # Gradient calculation.
      loss.backward()

      # Print loss every 10 iterations.
      if k%30==0:
          print('Loss {:.4f} at iter {:d}'.format(loss.item(),k))
          

      # Model weight modification based on the optimizer. 
      optimizer.step()
   ##################################################################

In [0]:
path_resnet = "gdrive/My Drive/projectA/Models/all_features_model_resnet"
model_resnet = torch.load(path)
path_densenet = "gdrive/My Drive/projectA/Models/all_features_model_densenet"
model_densenet = torch.load(path)
path_resnext = "gdrive/My Drive/projectA/Models/all_features_model_resnext"
model_resnext = torch.load(path)

In [0]:
correct_tot = np.zeros(12,)
total_tot = np.zeros(12,)
correct_tot1 = np.zeros(12,)
total_tot1 = np.zeros(12,)
correct = 0
total = 0
correct_1 = 0
total_1 = 0
num_of_larges = 0
with torch.no_grad():
      for data in test_loader:
          images, labels = data
          images = images.permute(0,3,1,2)
          images = images.to(device)
          labels = labels.to(device)
          outputs_resnet = torch.round(model_resnet(images).data + 0.473)
          outputs_resnext = torch.round(model_resnext(images).data + 0.478)
          outputs_densenet = torch.round(model_densenet(images).data + 0.476)
#           _, predicted = torch.max(outputs.data, 1)
          predicted = torch.round( (outputs_densenet + outputs_resnet + outputs_resnext)/float(3) )
          for i,line in enumerate(labels):
            for j,label in enumerate(line):
               total_tot[j] += 1
               correct_tot[j] += (predicted[i,j] == label).item()
               if label == 1:
                  total_tot1[j] += 1
                  correct_tot1[j] += (predicted[i,j] == label).item()
                  total_1 +=1
                  correct_1 += (predicted[i,j] == label).item()
          total += (labels.size(0) * labels.size(1) ) 
          correct += (predicted == labels).sum().item()            
print('Accuracy of the network on the 1795 test images: %d %%' % (100 * correct / total))
print(correct , "  " , total)
print('Accuracy of the network on the 1 class only: %d %%' % (100 * correct_1 / total_1))
print(correct_1 , "  " , total_1)



Accuracy of the network on the 1795 test images: 92 %
19864    21540
Accuracy of the network on the 1 class only: 62 %
266    429


# color | large vehicles

In [0]:
label_map_list = {
    0: [0], 1: [1, 6], 2: [2], 3: [1, 2, 3, 4, 5, 6, 7],
    4: [4, 5], 5: [4, 5], 6: [6, 7, 1], 7: [6, 7]
}

label_map = {
    'green': 0, 'yellow': 1, 'red': 2, 'other': 3,
    'blue': 4, 'black':5, 'white':6, 'silver/grey':7
}

def get_labels(annotationDB , recIndex):
  label = annotationDB.loc[recIndex, 'color']
  return label_map[label]


testFilename="gdrive/My Drive/projectA/Data/test_without_zoomout_large_vehicles.csv"
annotationFilename = "gdrive/My Drive/projectA/Data/train_without_zoomout_large_vehicles.csv"

def prepareInputs(annotationFilename, imgsFilePath = "gdrive/My Drive/projectA/Data/training_imagery/",
                  sampleShape = (128,128), maxSetSize = 10000):
    print("Start preproccesing")
    annotationDB = pd.read_csv(annotationFilename)
    nRecords = annotationDB.shape[0]
    Samples = {};
    Labels = {};
    area_per_size = []; maxDistances =[]; i=0
    numOfSamples = 0
    invalids = 0  
    for recIndex in range(nRecords):
        if maxSetSize < 0 or numOfSamples < maxSetSize:
            imgID = annotationDB.loc[recIndex, 'image_id']
            objectBoundingBox = (np.array([[annotationDB.loc[recIndex, 'p1_x'], annotationDB.loc[recIndex, 'p_1y']],
                                          [annotationDB.loc[recIndex, 'p2_x'], annotationDB.loc[recIndex, 'p2_y']],
                                          [annotationDB.loc[recIndex, 'p3_x'], annotationDB.loc[recIndex, 'p3_y']],
                                          [annotationDB.loc[recIndex, 'p4_x'], annotationDB.loc[recIndex, 'p4_y']]]))
            for i,point in enumerate(objectBoundingBox):
              for j,coord in enumerate(point):
                if coord < 0:
                  objectBoundingBox[i][j] = 0
                  
            imgFile = glob.glob(os.path.join(imgsFilePath, str(int(imgID)) + ".*"))
            if len(imgFile) == 1:
                img = cv2.imread(imgFile[0],cv2.IMREAD_COLOR)  # now img is 3-dim array
                cnt = objectBoundingBox.astype(int)
                patch = img[int(np.min(objectBoundingBox[:,1])):int(np.max(objectBoundingBox[:,1])),
                        int(np.min(objectBoundingBox[:,0])):int(np.max(objectBoundingBox[:,0]))]
                patchShape = patch.shape 
                if patchShape[0] > 0 and patchShape[1] > 0:
                    rect = cv2.minAreaRect(cnt)
                    box = cv2.boxPoints(rect)
                    box = np.int0(box)
                    cv2.drawContours(img, [box], 0, (0, 10, 0), 0)
                    img_crop, img_rot = crop_rect(img, rect)
                    patch = img_crop
                    patchShape =  patch.shape
                    
                    desired_size = sampleShape[0]
                    old_size = patchShape
                    ratio = float(sampleShape[0])/max(old_size)
                    new_size = tuple([int(x*ratio) for x in old_size])
                    old_sample = cv2.resize(patch, (new_size[1], new_size[0]))
                    delta_w = desired_size - new_size[1]
                    delta_h = desired_size - new_size[0]
                    top, bottom = delta_h//2, delta_h-(delta_h//2)
                    left, right = delta_w//2, delta_w-(delta_w//2)
                    color = [0, 0, 0]
                    sample = cv2.copyMakeBorder(old_sample, top, bottom, left, right, cv2.BORDER_CONSTANT,
                        value=color)
                    Samples[annotationDB.loc[recIndex, 'tag_id']] = sample
                    
                    Labels[annotationDB.loc[recIndex, 'tag_id']] = get_labels(annotationDB , recIndex)
                    numOfSamples += 1
                    
                    if annotationDB.loc[recIndex, 'color'] != 'white':
                      img_90 = np.rot90(sample)
                      new_tag =annotationDB.loc[recIndex, 'tag_id']*10+1
                      Samples[new_tag] = img_90
                      Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                      img_180 = np.rot90(img_90)
                      new_tag +=1
                      Samples[new_tag] = img_180
                      Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                      img_270 = np.rot90(img_180)
                      new_tag +=1
                      Samples[new_tag] = img_270
                      Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                      vertical_img = cv2.flip( sample, 1 )
                      new_tag +=1
                      Samples[new_tag] = vertical_img
                      Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                      vertical_img_90 = np.rot90(vertical_img)
                      new_tag +=1
                      Samples[new_tag] = vertical_img_90
                      Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                      vertical_img_180 = np.rot90(vertical_img_90)
                      new_tag +=1
                      Samples[new_tag] = vertical_img_180
                      Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                      vertical_img_270 = np.rot90(vertical_img_180)
                      new_tag +=1
                      Samples[new_tag] = vertical_img_270
                      Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                      numOfSamples += 7

                else:
                    print("Invalid image\n")
                    print (patchShape , "no. : " , recIndex , "tag: " , annotationDB.loc[recIndex, 'tag_id'])
                    invalids += 1
            else:
                pdb.set_trace()
                print("Image file missing!\n")
            i+=1
            if i%300 == 0:
              print("Finished image no. : " , i)
    print("Finished! valids: " , len(Samples) , "invalids: " , invalids)
    return [Samples, Labels, numOfSamples ]

In [0]:
'''dataset preperation for subclss problem'''
import torch
import torchvision
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as data

class DataSet_train(data.Dataset):
    def __init__(self, filename):
        self.DB = pd.read_csv(filename)
        data, labels, numOfSamples = prepareInputs(filename)
        self.data = list(data.values())
        self.target = list(labels.values())
        self.n_samples = numOfSamples
    
    def __len__(self):   # Length of the dataset.
        return self.n_samples
    
    def __getitem__(self, index):   # Function that returns one point and one label.
        return torch.Tensor(self.data[index].copy()),self.target[index]
      
class DataSet_test(data.Dataset):
    def __init__(self, filename):
        self.DB = pd.read_csv(filename)
        data, labels, numOfSamples = prepareInputs(filename)
        self.data = list(data.values())
        self.target = list(labels.values())
        self.n_samples = numOfSamples
    
    def __len__(self):   # Length of the dataset.
        return self.n_samples
    
    def __getitem__(self, index):   # Function that returns one point and one label.
        return torch.Tensor(self.data[index].copy()), self.target[index]


In [0]:
my_data = DataSet_train(annotationFilename)
batch_size = 50
my_loader = data.DataLoader(my_data,batch_size=batch_size,shuffle=True,num_workers=1)

In [0]:
test_data = DataSet_test(testFilename)
batch_size = 50
test_loader = data.DataLoader(test_data,batch_size=batch_size,shuffle=False,num_workers=1)

In [0]:
'''model definition'''
torch.cuda.empty_cache()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import torchvision.models as models
import torch.optim as optim
weights = torch.Tensor([3.,3.,3.,3.,3.,3.,1.,3.])
weights = weights.to(device)
model = torchvision.models.resnet18(pretrained=True)

model.fc = nn.Linear(512, 8)
model = model.to(device)
criterium = nn.CrossEntropyLoss(weight=weights)
optimizer = torch.optim.Adam(model.parameters(),lr=5e-4,weight_decay=1e-4)
best = 0

In [0]:
optimizer = torch.optim.Adam(model.parameters(),lr=1e-5,weight_decay=1e-4)

In [0]:
'''Training'''
'''TODO: try train in balanced set by augment only big cars'''
torch.cuda.empty_cache()
for epoch in range(500):
  print('Starting epoch number: ',epoch)
  ################## evaluation (testing) section:     ############
  if epoch > 0 :
    cnt_of_success = np.zeros(8,)
    sums = np.zeros(8,)
    correct = 0
    total = 0
    num_of_larges = 0
    with torch.no_grad():
      for data in test_loader:
          images, labels = data
          images = images.permute(0,3,1,2)
          images = images.to(device)
          labels = labels.to(device)
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          for i in range(len(predicted)):
            correct += (predicted[i].item() in label_map_list[labels.long().reshape(labels.__len__())[i].item()])
            sums[labels.long().reshape(labels.__len__())[i]]+=1
            

    print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))
    print(correct , "  " , total)

    if correct > best:
      best = correct
      path = "gdrive/My Drive/projectA/Models/color_large_model_resnet18"
      torch.save(model, path)
      print("model saved...")

     
  ##################################################################

  ######################### Training section:     ##################
  for k, (data, label) in enumerate(my_loader):
      # Definition of inputs as variables for the net.
      # requires_grad is set False because we do not need to compute the 
      # derivative of the inputs
      data   = Variable(data,requires_grad=False)
      label = Variable(label,requires_grad=False)
      data = data.permute(0,3,1,2)
      data = data.to(device)
      label = label.to(device)
      # Set gradient to 0.
      optimizer.zero_grad()
      # Feed forward.
      pred = model(data)
      pred = pred.to(device)

      # Loss calculation.
      loss = criterium(pred,label.view(-1))
      # Gradient calculation.
      loss.backward()

      # Print loss every 10 iterations.
      if k%30==0:
          print('Loss {:.4f} at iter {:d}'.format(loss.item(),k))
          

      # Model weight modification based on the optimizer. 
      optimizer.step()
   ##################################################################

In [0]:
path_resnet = "gdrive/My Drive/projectA/Models/all_features_model_resnet"
model_resnet = torch.load(path)
path_densenet = "gdrive/My Drive/projectA/Models/all_features_model_densenet"
model_densenet = torch.load(path)
path_resnext = "gdrive/My Drive/projectA/Models/all_features_model_resnext"
model_resnext = torch.load(path)

In [0]:
correct_tot = np.zeros(12,)
total_tot = np.zeros(12,)
correct_tot1 = np.zeros(12,)
total_tot1 = np.zeros(12,)
correct = 0
total = 0
correct_1 = 0
total_1 = 0
num_of_larges = 0
with torch.no_grad():
      for data in test_loader:
          images, labels = data
          images = images.permute(0,3,1,2)
          images = images.to(device)
          labels = labels.to(device)
          outputs_resnet = torch.round(model_resnet(images).data + 0.473)
          outputs_resnext = torch.round(model_resnext(images).data + 0.478)
          outputs_densenet = torch.round(model_densenet(images).data + 0.476)
          predicted = torch.round( (outputs_densenet + outputs_resnet + outputs_resnext)/float(3) )
          for i,line in enumerate(labels):
            for j,label in enumerate(line):
               total_tot[j] += 1
               correct_tot[j] += (predicted[i,j] == label).item()
               if label == 1:
                  total_tot1[j] += 1
                  correct_tot1[j] += (predicted[i,j] == label).item()
                  total_1 +=1
                  correct_1 += (predicted[i,j] == label).item()
          total += (labels.size(0) * labels.size(1) ) 
          correct += (predicted == labels).sum().item()            
print('Accuracy of the network on the 1795 test images: %d %%' % (100 * correct / total))
print(correct , "  " , total)
print('Accuracy of the network on the 1 class only: %d %%' % (100 * correct_1 / total_1))
print(correct_1 , "  " , total_1)



# color | small vehicles

In [0]:
label_map_list = {
    0: [0], 1: [1, 6], 2: [2], 3: [1, 2, 3, 4, 5, 6, 7],
    4: [4, 5], 5: [4, 5], 6: [6, 7, 1], 7: [6, 7]
}

label_map = {
    'green': 0, 'yellow': 1, 'red': 2, 'other': 3,
    'blue': 4, 'black':5, 'white':6, 'silver/grey':7
}

def get_labels(annotationDB , recIndex):
  label = annotationDB.loc[recIndex, 'color']
  return label_map[label]


testFilename="gdrive/My Drive/projectA/Data/test_without_zoomout_small_vehicles.csv"
annotationFilename = "gdrive/My Drive/projectA/Data/train_without_zoomout_small_vehicles.csv"

def prepareInputs(annotationFilename, imgsFilePath = "gdrive/My Drive/projectA/Data/training_imagery/",
                  sampleShape = (128,128), maxSetSize = 100000):
    print("Start preproccesing")
    annotationDB = pd.read_csv(annotationFilename)
    nRecords = annotationDB.shape[0]
    Samples = {};
    Labels = {};
    area_per_size = []; maxDistances =[]; i=0
    numOfSamples = 0
    invalids = 0  
    for recIndex in range(nRecords):
        if maxSetSize < 0 or numOfSamples < maxSetSize:
            imgID = annotationDB.loc[recIndex, 'image_id']
            objectBoundingBox = (np.array([[annotationDB.loc[recIndex, 'p1_x'], annotationDB.loc[recIndex, 'p_1y']],
                                          [annotationDB.loc[recIndex, 'p2_x'], annotationDB.loc[recIndex, 'p2_y']],
                                          [annotationDB.loc[recIndex, 'p3_x'], annotationDB.loc[recIndex, 'p3_y']],
                                          [annotationDB.loc[recIndex, 'p4_x'], annotationDB.loc[recIndex, 'p4_y']]]))
            for i,point in enumerate(objectBoundingBox):
              for j,coord in enumerate(point):
                if coord < 0:
                  objectBoundingBox[i][j] = 0
                  
            imgFile = glob.glob(os.path.join(imgsFilePath, str(int(imgID)) + ".*"))
            if len(imgFile) == 1:
                img = cv2.imread(imgFile[0],cv2.IMREAD_COLOR)  # now img is 3-dim array
                cnt = objectBoundingBox.astype(int)
                patch = img[int(np.min(objectBoundingBox[:,1])):int(np.max(objectBoundingBox[:,1])),
                        int(np.min(objectBoundingBox[:,0])):int(np.max(objectBoundingBox[:,0]))]
                patchShape = patch.shape 
                if patchShape[0] > 0 and patchShape[1] > 0:
                    rect = cv2.minAreaRect(cnt)
                    box = cv2.boxPoints(rect)
                    box = np.int0(box)
                    cv2.drawContours(img, [box], 0, (0, 10, 0), 0)
                    img_crop, img_rot = crop_rect(img, rect)
                    patch = img_crop
                    patchShape =  patch.shape
                    
                    desired_size = sampleShape[0]
                    old_size = patchShape
                    ratio = float(sampleShape[0])/max(old_size)
                    new_size = tuple([int(x*ratio) for x in old_size])
                    old_sample = cv2.resize(patch, (new_size[1], new_size[0]))
                    delta_w = desired_size - new_size[1]
                    delta_h = desired_size - new_size[0]
                    top, bottom = delta_h//2, delta_h-(delta_h//2)
                    left, right = delta_w//2, delta_w-(delta_w//2)
                    color = [0, 0, 0]
                    sample = cv2.copyMakeBorder(old_sample, top, bottom, left, right, cv2.BORDER_CONSTANT,
                        value=color)
                    Samples[annotationDB.loc[recIndex, 'tag_id']] = sample
                    
                    Labels[annotationDB.loc[recIndex, 'tag_id']] = get_labels(annotationDB , recIndex)
                    numOfSamples += 1
                    
                    if annotationDB.loc[recIndex, 'color'] not in ['white','silver/grey']:
                      img_90 = np.rot90(sample)
                      new_tag =annotationDB.loc[recIndex, 'tag_id']*10+1
                      Samples[new_tag] = img_90
                      Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                      img_180 = np.rot90(img_90)
                      new_tag +=1
                      Samples[new_tag] = img_180
                      Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                      img_270 = np.rot90(img_180)
                      new_tag +=1
                      Samples[new_tag] = img_270
                      Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                      if annotationDB.loc[recIndex, 'color'] not in ['black','blue']:
                        vertical_img = cv2.flip( sample, 1 )
                        new_tag +=1
                        Samples[new_tag] = vertical_img
                        Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                        vertical_img_90 = np.rot90(vertical_img)
                        new_tag +=1
                        Samples[new_tag] = vertical_img_90
                        Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                        vertical_img_180 = np.rot90(vertical_img_90)
                        new_tag +=1
                        Samples[new_tag] = vertical_img_180
                        Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                        vertical_img_270 = np.rot90(vertical_img_180)
                        new_tag +=1
                        Samples[new_tag] = vertical_img_270
                        Labels[new_tag] = Labels[annotationDB.loc[recIndex, 'tag_id']]
                        numOfSamples += 7
                      else:
                        numOfSamples += 3

                else:
                    print("Invalid image\n")
                    print (patchShape , "no. : " , recIndex , "tag: " , annotationDB.loc[recIndex, 'tag_id'])
                    invalids += 1
            else:
                pdb.set_trace()
                print("Image file missing!\n")
            i+=1
            if i%300 == 0:
              print("Finished image no. : " , i)
    print("Finished! valids: " , len(Samples) , "invalids: " , invalids)
    return [Samples, Labels, numOfSamples ]

In [0]:
'''dataset preperation for subclss problem'''
import torch
import torchvision
import torchvision.transforms as transforms
from PIL import Image
import matplotlib.pyplot as plt
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as data

class DataSet_train(data.Dataset):
    def __init__(self, filename):
        self.DB = pd.read_csv(filename)
        data, labels, numOfSamples = prepareInputs(filename)
        self.data = list(data.values())
        self.target = list(labels.values())
        self.n_samples = numOfSamples
    
    def __len__(self):   # Length of the dataset.
        return self.n_samples
    
    def __getitem__(self, index):   # Function that returns one point and one label.
        return torch.Tensor(self.data[index].copy()),self.target[index]
      
class DataSet_test(data.Dataset):
    def __init__(self, filename):
        self.DB = pd.read_csv(filename)
        data, labels, numOfSamples = prepareInputs(filename)
        self.data = list(data.values())
        self.target = list(labels.values())
        self.n_samples = numOfSamples
    
    def __len__(self):   # Length of the dataset.
        return self.n_samples
    
    def __getitem__(self, index):   # Function that returns one point and one label.
        return torch.Tensor(self.data[index].copy()), self.target[index]


In [0]:
my_data = DataSet_train(annotationFilename)
batch_size = 100
my_loader = data.DataLoader(my_data,batch_size=batch_size,shuffle=True,num_workers=1)

In [0]:
test_data = DataSet_test(testFilename)
batch_size = 100
test_loader = data.DataLoader(test_data,batch_size=batch_size,shuffle=False,num_workers=1)

In [0]:
'''model definition'''
torch.cuda.empty_cache()
# device = torch.device("cpu")
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import torchvision.models as models
import torch.optim as optim
weights = torch.Tensor([8.,8.,1.5,1.,1.5,1.,1.,1.])
weights = weights.to(device)
model = torchvision.models.resnet18(pretrained=True)

model.fc = nn.Linear(512, 8)
model = model.to(device)
criterium = nn.CrossEntropyLoss(weight=weights)
optimizer = torch.optim.Adam(model.parameters(),lr=5e-4,weight_decay=1e-4)
best = 0

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
optimizer = torch.optim.Adam(model.parameters(),lr=2e-3,weight_decay=1e-4)
path_resnet = "gdrive/My Drive/projectA/Models/all_features_large_model_resnet"
model = torch.load(path_resnet)

In [0]:
'''Training'''
'''TODO: try train in balanced set by augment only big cars'''
best = 0
torch.cuda.empty_cache()
for epoch in range(100):
  print('Starting epoch number: ',epoch)
  ################## evaluation (testing) section:     ############
  if epoch > 0 :
    cnt_of_success = np.zeros(8,)
    sums = np.zeros(8,)
    correct = 0
    total = 0
    num_of_larges = 0
    with torch.no_grad():
      for data in test_loader:
          images, labels = data
          images = images.permute(0,3,1,2)
          images = images.to(device)
          labels = labels.to(device)
          outputs = model(images)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          for i in range(len(predicted)):
            correct += (predicted[i].item() in label_map_list[labels.long().reshape(labels.__len__())[i].item()])
            sums[labels.long().reshape(labels.__len__())[i]]+=1
            

    print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))
    print(correct , "  " , total)
    print("predicted: ",predicted)
    print("GT: ", labels.long().reshape(labels.__len__()))
    if correct > best:
      best = correct
      path = "gdrive/My Drive/projectA/Models/color_small_model_resnet18"
      torch.save(model, path)
      print("model saved...")

    if correct > best:
      best = correct
      path = "gdrive/My Drive/projectA/Models/all_features_model"
      torch.save(model, path)
    if (100 * correct / total) > 95:
      break
    if (100 * correct / total) > 92 and (100 * correct_1 / total_1)>72:
      path = "gdrive/My Drive/projectA/Models/all_features_model_densenet"
      torch.save(model, path)
     
  ##################################################################

  ######################### Training section:     ##################
  for k, (data, label) in enumerate(my_loader):
      # Definition of inputs as variables for the net.
      # requires_grad is set False because we do not need to compute the 
      # derivative of the inputs
      data   = Variable(data,requires_grad=False)
      label = Variable(label,requires_grad=False)
      data = data.permute(0,3,1,2)
      data = data.to(device)
      label = label.to(device)
      # Set gradient to 0.
      optimizer.zero_grad()
      # Feed forward.
      pred = model(data)
      pred = pred.to(device)

      # Loss calculation.
      loss = criterium(pred,label.view(-1))
      # Gradient calculation.
      loss.backward()

      # Print loss every 10 iterations.
      if k%30==0:
          print('Loss {:.4f} at iter {:d}'.format(loss.item(),k))
          

      # Model weight modification based on the optimizer. 
      optimizer.step()
   ##################################################################

In [0]:
path_resnet = "gdrive/My Drive/projectA/Models/all_features_model_resnet"
model_resnet = torch.load(path)
path_densenet = "gdrive/My Drive/projectA/Models/all_features_model_densenet"
model_densenet = torch.load(path)
path_resnext = "gdrive/My Drive/projectA/Models/all_features_model_resnext"
model_resnext = torch.load(path)

In [0]:
correct_tot = np.zeros(12,)
total_tot = np.zeros(12,)
correct_tot1 = np.zeros(12,)
total_tot1 = np.zeros(12,)
correct = 0
total = 0
correct_1 = 0
total_1 = 0
num_of_larges = 0
with torch.no_grad():
      for data in test_loader:
          images, labels = data
          images = images.permute(0,3,1,2)
          images = images.to(device)
          labels = labels.to(device)
          outputs_resnet = torch.round(model_resnet(images).data + 0.473)
          outputs_resnext = torch.round(model_resnext(images).data + 0.478)
          outputs_densenet = torch.round(model_densenet(images).data + 0.476)
#           _, predicted = torch.max(outputs.data, 1)
          predicted = torch.round( (outputs_densenet + outputs_resnet + outputs_resnext)/float(3) )
          for i,line in enumerate(labels):
            for j,label in enumerate(line):
               total_tot[j] += 1
               correct_tot[j] += (predicted[i,j] == label).item()
               if label == 1:
                  total_tot1[j] += 1
                  correct_tot1[j] += (predicted[i,j] == label).item()
                  total_1 +=1
                  correct_1 += (predicted[i,j] == label).item()
          total += (labels.size(0) * labels.size(1) ) 
          correct += (predicted == labels).sum().item()            
#           print(predicted.sum() )
print('Accuracy of the network on the 1795 test images: %d %%' % (100 * correct / total))
print(correct , "  " , total)
print('Accuracy of the network on the 1 class only: %d %%' % (100 * correct_1 / total_1))
print(correct_1 , "  " , total_1)

